In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
import math
from numpy import dot
from numpy.linalg import norm

In [61]:
# 유튜브 구독정보 난수생성기로 임의 생성

class RandomGenerator(object):
   
    def __init__(self, random_state=1):
        """
        -매개변수-
        random_state : 가중치 초기화를 위한 무작위 난수 생성기의 seed
        """
        self.random_state = random_state
        
    def GeneratingRandomData(self, loc=4, size=5000, scale=3, n_features=10):
        """
        난수생성기로 임의 데이터 생성
        """
    
        np.random.seed(self.random_state)
        # 특정 seed를 가지는 np.random.RandomState() 객체를 만들고, 이 객체를 통해서 난수 생성
        self.data = np.random.normal(loc, scale, size)
        for i in range(len(self.data)):
            self.data[i] = (abs(int(self.data[i])))
                
        return self.data.reshape(-1, n_features)
    
    def GeneratingFeature(self): # feature 임의 생성
        self.feature = ['Game', 'Cook', "Travel", "Money", "Academy", "Learning", "Sing", "Sports", "Machine", "Mistery"]
    
    def ArrayToDataframe(self, random_array, n_sample):
        return pd.DataFrame(random_array, index=range(0, n_sample), columns=self.feature)

In [62]:
# 난수생성기 객체 생성

dataGenerator = RandomGenerator(156)

In [63]:
randomData = dataGenerator.GeneratingRandomData()
randomData.shape

(500, 10)

In [64]:
# feature 이름 리스트 구성

feature_names = ['Game', 'Cook', "Travel", "Money", "Academy", "Learning", "Sing", "Sports", "Machine", "Mistery"]

In [65]:
dataGenerator.GeneratingFeature()

In [66]:
dfRandomData = dataGenerator.ArrayToDataframe(randomData, 500)
dfRandomData

,Game,Cook,Travel,Money,Academy,Learning,Sing,Sports,Machine,Mistery
0,5.0,1.0,1.0,2.0,8.0,3.0,5.0,5.0,1.0,0.0
1,2.0,1.0,3.0,4.0,3.0,2.0,0.0,5.0,0.0,6.0
2,1.0,0.0,2.0,1.0,3.0,9.0,3.0,5.0,11.0,2.0
3,5.0,3.0,4.0,3.0,6.0,1.0,4.0,2.0,4.0,0.0
4,10.0,6.0,3.0,1.0,6.0,3.0,4.0,7.0,2.0,8.0
...,...,...,...,...,...,...,...,...,...,...
495,3.0,8.0,7.0,6.0,3.0,2.0,7.0,7.0,9.0,1.0
496,6.0,2.0,4.0,4.0,10.0,6.0,3.0,6.0,1.0,0.0
497,0.0,6.0,4.0,0.0,7.0,2.0,5.0,3.0,1.0,6.0
498,8.0,1.0,6.0,6.0,3.0,6.0,1.0,5.0,2.0,6.0


In [287]:
# cosine_similarity 원본

from sklearn.metrics.pairwise import euclidean_distances

class Classification(object):
    
    def __init__(self, df_data):
        self.df = df_data
    
    def Extraction(self):
        # 가장 선호하는 feature 추출
        self.ls = []
        for i in range(0, self.df.shape[0]):
            self.ls.append(self.df.sort_values(by=[i], axis=1).iloc[i, -2:].index.tolist())
            
        # 각 이용자의 구독 채널 수 추출
        arr = np.array(self.df)
        self.num_ch = np.sum(arr, axis=1).reshape(-1, 1)
        
    def Ohe(self): # 각 샘플에 대해 추출한 가장 선호하는 2개의 특성에 대해 원핫인코딩
        ohe = OneHotEncoder(sparse=False)
        return ohe.fit_transform(self.ls)
    
    def makeData(self, train_feature):
        self.X_train = np.hstack((self.num_ch, train_feature))
        
    def Kmeans(self): # k_means 클러스터링
        model = KMeans(n_clusters=16)
        model.fit(self.X_train)
        return np.array(model.predict(self.X_train))

    def cosine_similarity(self, pt1, pt2):
      #sj: cosine similarity를 사용하면 결이 비슷한 사용자를 찾을 수 있음.
      #sj: 구독 채널 Id는 관련성이 있을 수록 cosine similarity가 작음.
        return dot(pt1, pt2) / (norm(pt1) * norm(pt2))

    def ReturnResult_cosine_similarity(self, k=5):
        
        k_users = np.empty((0,k))
        for i in range(0, self.X_train.shape[0]):
            distance = []
            for j in range(0, self.X_train.shape[0]):
                if i != j:
                    distance.append(self.cosine_similarity(self.X_train[i, :], self.X_train[j, :]))
                if j == (self.X_train.shape[0] - 1):
                    # i 번째 데이터 포인터와 다른 데이터들 사이의 거리를 작은순으로 정렬한 후 해당 인덱스를 이용해서 가장 가까운 k명의 이용자 선별
                    distance_idx = np.array(distance).argsort()
                    k_users = np.append(k_users, [distance_idx[:k]], axis=0)
                    
        return k_users
        


In [279]:
# sklearn의 cosine_similarity 사용
# 코사인 유사도 계산 시 시간복잡도 문제 해결

from sklearn.metrics.pairwise import cosine_similarity

class Classification(object):
    
    def __init__(self, df_data):
        self.df = df_data
    
    def Extraction(self):
        # 가장 선호하는 feature 추출
        self.ls = []
        for i in range(0, self.df.shape[0]):
            self.ls.append(self.df.sort_values(by=[i], axis=1).iloc[i, -2:].index.tolist())
            
        # 각 이용자의 구독 채널 수 추출
        arr = np.array(self.df)
        self.num_ch = np.sum(arr, axis=1).reshape(-1, 1)
        
    def Ohe(self): # 각 샘플에 대해 추출한 가장 선호하는 2개의 특성에 대해 원핫인코딩
        ohe = OneHotEncoder(sparse=False)
        return ohe.fit_transform(self.ls)
    
    def makeData(self, train_feature):
        self.X_train = np.hstack((self.num_ch, train_feature))
        
    def Kmeans(self): # k_means 클러스터링
        model = KMeans(n_clusters=16)
        model.fit(self.X_train)
        return np.array(model.predict(self.X_train))

    def ReturnResult_cosine_similarity(self, k=5):
        
        k_users = np.empty((0,k))
        for i in range(0, self.X_train.shape[0]):
            #distance = []
            #distance.append(cosine_similarity(self.X_train[i].reshape(1,-1), self.X_train))
            distance = [cosine_similarity(self.X_train[i].reshape(1,-1), self.X_train)]
            
            # i 번째 데이터 포인터와 다른 데이터들 사이의 거리를 작은순으로 정렬한 후 해당 인덱스를 이용해서 가장 가까운 k명의 이용자 선별
            distance_idx = np.array(distance).ravel().argsort()
            k_users = np.append(k_users, [distance_idx[:k]], axis=0)
                    
        return k_users
    
   # def sendResult(self, user, k_users):
    #    result = {user: k_users[]}

In [288]:
cl = Classification(dfRandomData)

In [289]:
cl.Extraction()
cl.ls

[['Sports', 'Academy'],
 ['Sports', 'Mistery'],
 ['Learning', 'Machine'],
 ['Game', 'Academy'],
 ['Mistery', 'Game'],
 ['Academy', 'Travel'],
 ['Travel', 'Sports'],
 ['Machine', 'Mistery'],
 ['Game', 'Academy'],
 ['Machine', 'Academy'],
 ['Game', 'Learning'],
 ['Travel', 'Money'],
 ['Game', 'Travel'],
 ['Sports', 'Learning'],
 ['Learning', 'Sports'],
 ['Money', 'Machine'],
 ['Learning', 'Sing'],
 ['Learning', 'Sing'],
 ['Mistery', 'Sports'],
 ['Learning', 'Sing'],
 ['Academy', 'Machine'],
 ['Mistery', 'Sing'],
 ['Money', 'Academy'],
 ['Mistery', 'Travel'],
 ['Learning', 'Academy'],
 ['Learning', 'Machine'],
 ['Learning', 'Sing'],
 ['Travel', 'Machine'],
 ['Mistery', 'Sports'],
 ['Sing', 'Game'],
 ['Machine', 'Academy'],
 ['Mistery', 'Sports'],
 ['Cook', 'Travel'],
 ['Travel', 'Game'],
 ['Sing', 'Sports'],
 ['Game', 'Sports'],
 ['Cook', 'Mistery'],
 ['Money', 'Machine'],
 ['Cook', 'Money'],
 ['Money', 'Sing'],
 ['Travel', 'Sing'],
 ['Machine', 'Money'],
 ['Game', 'Cook'],
 ['Travel', 'L

In [290]:
train_feature = cl.Ohe()
train_feature

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.]])

In [291]:
cl.makeData(train_feature)

In [292]:
result = cl.Kmeans()
result

array([15,  1,  0,  8, 13,  7, 15, 10,  9, 12,  4,  7,  7, 10,  4, 15,  4,
        9,  8, 15,  7, 12, 11,  8,  3, 15,  8,  1,  2,  3,  2,  4,  9,  0,
       13, 12,  2,  1,  2, 15,  1, 15,  9,  1,  9,  0,  0,  8,  9,  8,  3,
        4,  7,  7,  0, 11,  7, 11,  4,  1,  7,  1,  6,  4,  5,  4,  3,  5,
        0, 10,  4, 10, 13,  0,  4,  1,  9,  8,  7, 15,  9, 14, 12,  0,  8,
        2, 11, 13, 12,  1,  8,  2, 12, 11,  4, 11,  8,  9,  3,  3,  8, 12,
       12,  9, 13,  1,  8,  8,  0, 10,  9,  8,  7,  9,  2,  2,  0, 15,  8,
        1, 10,  3, 15, 11,  2, 15,  9, 15,  7,  3,  7, 13,  8, 15,  8,  9,
       11,  1,  7, 13,  7,  6,  4,  0,  7,  0, 13,  9, 11,  2,  4,  9, 13,
        0, 11, 12,  0, 13,  9,  0, 12, 12, 13, 12, 13,  9,  2,  9,  7, 13,
        2,  1, 13,  7,  7,  9,  8, 15,  8,  0,  0,  3, 10,  1,  9,  0,  9,
        9,  5,  9,  0,  0,  9, 11,  9,  7,  9,  0, 13,  0,  2,  0,  9,  6,
        7,  1, 10,  7, 12,  0, 13, 12,  7,  7,  8,  1,  0, 11, 13,  0,  4,
        5,  6,  8, 10, 12

In [293]:
k_users = cl.ReturnResult_cosine_similarity()

In [286]:
k_users

array([[455., 364., 290.,  62., 379.],
       [455., 364.,  62., 290., 397.],
       [364., 455.,  62., 290., 379.],
       ...,
       [364., 290.,  62., 397., 379.],
       [364., 290.,  62., 379., 397.],
       [ 62., 393., 397., 379., 222.]])

In [294]:
k_users

array([[363., 454., 289.,  61., 392.],
       [363., 454.,  61., 289., 392.],
       [363., 454., 289.,  61., 378.],
       ...,
       [364., 290.,  62., 397., 379.],
       [364., 290.,  62., 379., 397.],
       [ 62., 379., 397., 393., 141.]])